In [1]:
import requests as req
from bs4 import BeautifulSoup as bs

In [2]:
import sys
from src.functions import * 

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_countries_by_meat_consumption"
tabla = scraping(url)


In [4]:
filas = filas(tabla)

In [5]:
import pandas as pd

col_names=["country", "kg/person_2002", "kg/person_2009", "kg/person_2017", "kg/person_2017_2"]

data=filas[1:]

meat_consumption=pd.DataFrame(data, columns=col_names)
meat_consumption = meat_consumption.drop(['kg/person_2017_2'], axis=1)
meat_consumption.to_csv("Data/meat_consumption.csv", index=False)

In [6]:
meat_consumption["kg/person_2009"].fillna("unknown", inplace = True)
meat_consumption = meat_consumption[meat_consumption['kg/person_2017'].notna()]
meat_consumption.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163 entries, 0 to 184
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   country         163 non-null    object
 1   kg/person_2002  163 non-null    object
 2   kg/person_2009  163 non-null    object
 3   kg/person_2017  163 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB


In [7]:
alcohol_by_country = pd.read_csv("Data/alcohol_by_country.csv")
obesity_by_country = pd.read_csv("Data/obesity_by_country.csv")

In [8]:

meat_consumption = meat_consumption[~meat_consumption["kg/person_2017"].str.contains("]")]


In [9]:
meat_consumption[["kg/person_2017"]] = meat_consumption[["kg/person_2017"]].astype(float)

In [10]:
alcohol_by_country["country"] = alcohol_by_country["country"].replace(["United States"], "United States of America")

meat_consumption = meat_consumption.sort_values("kg/person_2017", ascending=False)
meat_consumption.head(15)

,country,kg/person_2002,kg/person_2009,kg/person_2017
176,United States of America,124.8,120.2,121.00
7,Australia,108.2,111.5,114.26
5,Argentina,79.7,98.3,112.17
22,Brazil,82.4,85.3,99.36
79,Israel,97.1,96,97.82
141,Samoa,82.6,87.2,97.76
140,Saint Vincent and the Grenadines,79.1,91.4,93.38
139,Saint Lucia,124.1,93.6,91.82
132,Portugal,91.1,93.4,90.99
28,Canada,108.1,94.3,90.09


In [11]:
meat_consumption_mysample = meat_consumption[:15]
meat_consumption_mysample

,country,kg/person_2002,kg/person_2009,kg/person_2017
176,United States of America,124.8,120.2,121.00
7,Australia,108.2,111.5,114.26
5,Argentina,79.7,98.3,112.17
22,Brazil,82.4,85.3,99.36
79,Israel,97.1,96,97.82
141,Samoa,82.6,87.2,97.76
140,Saint Vincent and the Grenadines,79.1,91.4,93.38
139,Saint Lucia,124.1,93.6,91.82
132,Portugal,91.1,93.4,90.99
28,Canada,108.1,94.3,90.09


In [12]:
dict_col = {"Unnamed: 0": "country", "Both.sexes": "both.sexes", "Male": "male", "Female": "female", "country": "x"}
obesity_by_country.rename(columns=dict_col, inplace=True)
#obesity_by_country = obesity_by_country.drop(['x'], axis=1)
obesity_by_country = obesity_by_country.sort_values('both.sexes',ascending=False)
obesity_by_country["country"] = obesity_by_country["country"].replace(["United Kingdom of Great Britain and Northern Ireland"], "United Kingdom")
obesity_by_country.head()

,country,both.sexes,male,female
119,Nauru,60.7,58.5,63.0
39,Cook Islands,55.3,52.0,58.6
130,Palau,54.9,51.3,58.4
107,Marshall Islands,52.4,47.9,56.9
180,Tuvalu,51.0,46.4,55.6


In [13]:
alcohol_by_country_2 = alcohol_by_country[["country", "alcconsumption"]]
alcohol_by_country_2 = alcohol_by_country_2.sort_values('alcconsumption',ascending=False)
alcohol_by_country_2.head()

,country,alcconsumption
126,Moldova,23.01
100,"Korea, Rep.",19.15
16,Belarus,18.85
200,Ukraine,17.47
59,Estonia,17.24


In [14]:
study = meat_consumption.merge(alcohol_by_country_2, left_on='country', right_on='country')
study.head()

,country,kg/person_2002,kg/person_2009,kg/person_2017,alcconsumption
0,United States of America,124.8,120.2,121.00,9.70
1,Australia,108.2,111.5,114.26,10.21
2,Argentina,79.7,98.3,112.17,9.35
3,Brazil,82.4,85.3,99.36,10.08
4,Israel,97.1,96,97.82,2.52


In [15]:
study_2 = study.merge(obesity_by_country, left_on='country', right_on='country')
study_2 = study_2[["country", "kg/person_2017", "alcconsumption", "both.sexes", "male", "female"]]

In [16]:
study_2 = study_2.sort_values('kg/person_2017',ascending=False)
country_meat = study_2[["country", "kg/person_2017"]]
country_alcohol = study_2[["country", "alcconsumption"]]
country_obesity = study_2[["country", "both.sexes", "male", "female"]]
study_2.head()

,country,kg/person_2017,alcconsumption,both.sexes,male,female
0,United States of America,121.00,9.70,37.3,36.5,38.2
1,Australia,114.26,10.21,30.4,30.7,30.1
2,Argentina,112.17,9.35,28.5,27.4,29.6
3,Brazil,99.36,10.08,22.3,18.5,25.9
4,Israel,97.82,2.52,26.7,26.1,27.2


In [23]:
country_obesity[["both.sexes",	"male",	"female"]] = country_obesity[["both.sexes", "male", "female"]].astype(float)
country_alcohol[["alcconsumption"]] = country_alcohol[["alcconsumption"]].astype(float)

In [24]:
country_meat.to_csv("Data/country_meat.csv", index=False)
country_alcohol.to_csv("Data/country_alcohol.csv", index=False)
country_obesity.to_csv("Data/country_obesity.csv", index=False)
study_2.to_csv("Data/study_2.csv", index=False)